<a href="https://colab.research.google.com/github/vishkaush/misc-projects/blob/main/TensorFlowv1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Getting Comfortable

In [12]:
import numpy as np
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

In [17]:
coefficients = np.array([[1.], [-10.], [25.]])
w = tf.Variable(0,dtype=tf.float32)
x = tf.placeholder(tf.float32, [3,1])  #to be used s coefficients of the cost functions
#cost = tf.add(tf.add(w**2, tf.multiply(-10.,w)),25)   # J(w) = w^2-10w+25 = (w-5)^2
#cost = w**2 - 10*w + 25 
cost = x[0][0]*w**2 + x[1][0]*w + x[2][0]  #this is the heart of the program - internally builds a computation graph
train = tf.train.GradientDescentOptimizer(0.01).minimize(cost) # specifies the algo, automatically computes the gradients
init = tf.global_variables_initializer()
session = tf.Session()
session.run(init)
print(session.run(w))
#above three lines can be replaced by the following three lines
# with tf.Session() as session:
#   session.run(init)
#   print(session.run(w))

0.0


In [18]:
session.run(train, feed_dict={x:coefficients})
print(session.run(w))

0.099999994


In [19]:
for i in range(1000):
  session.run(train, feed_dict={x:coefficients})
print(session.run(w))

4.999988
